<a href="https://colab.research.google.com/github/FrankPerera04/DSPL_Individual_CW/blob/main/Developing%20Streamlit/Setting_up_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Instructions for Usage**
1. Install in packages
2. Write the App
3. Start streamlit and tunnel
4. Get the ipv4 code
5. Run and get the Tunnel link
6. Enter the ipv4 code into the tunnel website to access streamlit

In [ ]:
!pip install streamlit -q
!npm install -g localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.0 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼
added 22 packages in 3s
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼

In [ ]:
# Write the app
%%writefile app.py
# Import necessary libraries
import streamlit as st
import pandas as pd
import plotly.express as px

# Set page configuration
st.set_page_config(page_title="Accommodation Dashboard", layout="wide")

# Load data
def load_data():
    file_path = "https://raw.githubusercontent.com/FrankPerera04/DSPL_Individual_CW/refs/heads/main/processed_accommodation_data%20(1).csv"
    df = pd.read_csv(file_path)
    return df

df = load_data()

# Sidebar Navigation
st.sidebar.header("🏠 Navigation")
page = st.sidebar.radio(
    "Go to",
    ("About",
     "Accommodation Capacity by Region",
     "Geographic Gaps",
     "Type Distribution",
     "Action Signals")
)

# Sidebar Filters
st.sidebar.header("🔎 Filter Options")

# District filter (multi-select)
all_districts = sorted(df["District"].dropna().unique().tolist())
selected_districts = st.sidebar.multiselect("Select District(s)", options=["All"] + all_districts, default=["All"])

# Accommodation Type filter (multi-select)
all_types = sorted(df["Type"].dropna().unique().tolist())
selected_types = st.sidebar.multiselect("Select Accommodation Type(s)", options=["All"] + all_types, default=["All"])

# Apply filters
filtered_df = df.copy()
if "All" not in selected_districts:
    filtered_df = filtered_df[filtered_df["District"].isin(selected_districts)]
if "All" not in selected_types:
    filtered_df = filtered_df[filtered_df["Type"].isin(selected_types)]

# Main Page Routing
if page == "About":
    st.title("🏨 Accommodation Dashboard")
    st.write("Welcome to the Accommodation Insights Dashboard! Navigate using the sidebar.")

elif page == "Accommodation Capacity by Region":
    st.title("🏙️ Accommodation Capacity by Region")

    # Total number of accommodations per district
    st.subheader("Total Number of Accommodations per District")
    accommodations_per_district = filtered_df.groupby("District")["Name"].count().reset_index().sort_values(by="Name", ascending=False)
    fig1 = px.bar(accommodations_per_district, x="District", y="Name", color="Name", text="Name", template="plotly_dark")
    fig1.update_layout(xaxis_title="District", yaxis_title="Total Accommodations", xaxis_tickangle=-45)
    st.plotly_chart(fig1, use_container_width=True)

    # Total number of rooms per district
    st.subheader("Total Number of Rooms per District")
    rooms_per_district = filtered_df.groupby("District")["Rooms"].sum().reset_index().sort_values(by="Rooms", ascending=False)
    fig2 = px.bar(rooms_per_district, x="District", y="Rooms", color="Rooms", text="Rooms", template="plotly_dark")
    fig2.update_layout(xaxis_title="District", yaxis_title="Total Rooms", xaxis_tickangle=-45)
    st.plotly_chart(fig2, use_container_width=True)

    # Average size of accommodations
    st.subheader("Average Size of Accommodations (Rooms per Property)")
    avg_rooms_per_property = (filtered_df.groupby("District")["Rooms"].mean()).reset_index().sort_values(by="Rooms", ascending=False)
    fig3 = px.bar(avg_rooms_per_property, x="District", y="Rooms", color="Rooms", text=avg_rooms_per_property["Rooms"].round(1), template="plotly_dark")
    fig3.update_layout(xaxis_title="District", yaxis_title="Average Rooms per Property", xaxis_tickangle=-45)
    st.plotly_chart(fig3, use_container_width=True)

elif page == "Geographic Gaps":
    st.title("🗺️ Geographic Gaps")

    district_counts = filtered_df.groupby("District").agg(Total_Accommodations=("Name", "count")).reset_index()
    map_df = pd.merge(filtered_df, district_counts, on="District", how="left")

    st.map(map_df[["Latitude", "Logitiute"]].rename(columns={"Logitiute": "longitude", "Latitude": "latitude"}))

    low_accommodation_threshold = 5
    low_acc_districts = district_counts[district_counts["Total_Accommodations"] <= low_accommodation_threshold]

    if not low_acc_districts.empty:
        st.warning("⚠️ Districts with very few accommodations (<= 5):")
        st.dataframe(low_acc_districts, use_container_width=True)
    else:
        st.success("✅ All districts have sufficient accommodation coverage.")

elif page == "Type Distribution":
    st.title("🏘️ Accommodation Type Distribution")

    type_distribution = filtered_df["Type"].value_counts().reset_index()
    type_distribution.columns = ["Accommodation Type", "Count"]

    fig_type = px.pie(
        type_distribution,
        names="Accommodation Type",
        values="Count",
        title="Accommodation Types Share",
        hole=0.4,
        template="plotly_dark"
    )
    st.plotly_chart(fig_type, use_container_width=True)

    st.dataframe(type_distribution, use_container_width=True)

elif page == "Action Signals":
    st.title("🚨 Action Signals: Target for Improvements")

    low_threshold = 5
    action_df = filtered_df.groupby("District").agg(
        Total_Accommodations=("Name", "count"),
        Total_Rooms=("Rooms", "sum")
    ).reset_index()

    low_accommodation_districts = action_df[action_df["Total_Accommodations"] <= low_threshold]

    if not low_accommodation_districts.empty:
        st.warning(f"⚠️ {len(low_accommodation_districts)} District(s) have <= {low_threshold} accommodations. Consider investing here!")
        st.dataframe(low_accommodation_districts, use_container_width=True)

        fig_low = px.bar(
            low_accommodation_districts,
            x="District",
            y="Total_Accommodations",
            color="Total_Accommodations",
            text="Total_Accommodations",
            title="Districts with Low Number of Accommodations",
            template="plotly_dark"
        )
        fig_low.update_layout(xaxis_title="District", yaxis_title="Total Accommodations", xaxis_tickangle=-45)
        st.plotly_chart(fig_low, use_container_width=True)
    else:
        st.success("✅ All districts have sufficient number of accommodations!")


Writing app.py


In [ ]:
# Start Streamlit and tunnel in the background
import threading
import time

def run_app():
    get_ipython().system('streamlit run app.py &')

threading.Thread(target=run_app).start()
time.sleep(10)  # wait for Streamlit to start




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.57.220.32:8501



In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.57.220.32


In [ ]:
# Create the tunnel
get_ipython().system('npx localtunnel --port 8501')

⠙⠹⠸⠼⠴your url is: https://quiet-feet-like.loca.lt
────────────────────────── Traceback (most recent call last) ───────────────────────────
  /usr/local/lib/python3.11/dist-packages/streamlit/runtime/scriptrunner/exec_code.py:  
  121 in exec_func_with_error_handling                                                  
                                                                                        
  /usr/local/lib/python3.11/dist-packages/streamlit/runtime/scriptrunner/script_runner  
  .py:640 in code_to_exec                                                               
                                                                                        
  /content/app.py:15 in <module>                                                        
                                                                                        
     12 │   # Change this to your Google Drive path if needed                           
     13 │   return pd.read_csv("/content/drive/MyDrive/proce